# TP2 - Exploration de topics

<img src="./figures/figure2.png" width="1000">

# Latent Dirichlet Allocation (LDA)

La Latent Dirichlet Allocation (LDA) <span class="badge badge-secondary">(Blei et al., 2001)</span> est une méthode de <b>topic modelling</b>. C'est l'une des méthodes les plus utilisées dans ce domaine. La LDA prend en input une collection de documents et cherche à identifier les topics ou thèmes spécifiques dans l'ensemble du corpus.

<span class="badge badge-secondary">(Blei et al., 2001)</span> David M. Blei, Andrew Y. Ng, Michael I. Jordan: Latent Dirichlet Allocation. NIPS 2001: 601-608

Par exemple, on peut considérer le corpus suivant :

In [ ]:
# !pip install -q scikit-learn==0.23.2 pyLDAvis nltk==3.5 unidecode pysrt

# Only if running in colab
# !git clone https://github.com/AntoineSimoulin/m2-data-sciences.git

In [48]:
documents = [
    "Nous avons pris l'avion pour aller en vacances.",
    "J'ai fait de la plongée en vacances."
    "Nous avons joué au foot hier matin.",
    "J'ai pris le taxi.",
]

In [49]:
n_documents = len(documents)

L'objectif est d'identifier les thèmes ou <i>topics</i> qui correspondent à chaque phrase. Ici, on pourrait considérer des thèmes commes les <i>vacances</i>, le <i>sport</i> ou encore les <i>moyens de transports</i>.

La LDA est un outil qui permet de faire exactement ça. Il s'agit d'un modèle <b>génératif</b> qui cherche à expliquer une observation (ici le corpus) en s'appuyant sur des variables latentes (les topics). En plus de l'epxloration de thèmes, ce type de modèle peut être utilisé pour des tâches non supervisées, en particulier le clustering.

La LDA décrit chaque document comme une distribution de **topics**, eux mêmes caractérisés par une **distribution de mots**. En pratique, on va introduire une **variable latente** et exprimer chaque document comme une distribution de cette variable. Cette variable sera elle même décrite comme une distribution sur le vocabulaire comme décrit ci-dessous. Ainsi chaque topic n'est pas décrit explicitement. Il doit être interprété en fonction de sa distribution sur les mots du vocabulaire.

<img src="./figures/lda-idee.png" width="1000">



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [75]:
vectorizer = CountVectorizer()

vectorizer.fit(documents)

w2idx = {w: i for (i, w) in enumerate(vectorizer.get_feature_names())}
idx2w = {i: w for (i, w) in enumerate(vectorizer.get_feature_names())}

print(len(w2idx))

20


In [51]:
tokenizer = vectorizer.build_tokenizer()

In [124]:
documents = [[t.lower() for t in tokenizer(d)] for d in documents]
for doc in documents:
    print(doc)

['nous', 'avons', 'pris', 'avion', 'pour', 'aller', 'en', 'vacances']
['ai', 'fait', 'de', 'la', 'plongée', 'en', 'vacances', 'nous', 'avons', 'joué', 'au', 'foot', 'hier', 'matin']
['ai', 'pris', 'le', 'taxi']


In [53]:
vocab_size = len(vectorizer.get_feature_names())
print(vocab_size)

20


In [146]:
print("matrice d'occurence :")

n = np.zeros((len(documents), vocab_size), dtype=int)
for (doc_idx, doc) in enumerate(documents):
    words_idx, words_freq = np.unique(doc, return_counts=True)
    for (w, f) in zip(words_idx, words_freq):
        n[doc_idx][w2idx[w]] = f
print(n)

matrice d'occurence :
[[0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1]
 [1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0]]


## 1. Modélisation

La séance précédente, on a vu que l'on pouvait représenter les documents comme une distribution sur les mots. Les paramètres de cette distribution peuvent être calculés simplement en utilisant la fréquence des mots dans le document (le modèle Bag-Of_Word) ou alors en pondérant les mots en fonction de leur fréquence dans l'ensemble du corpus (modèle TF-IDF).

<img src="./figures/bow.png" width="500">

### Les modèles génératifs

La LDA est un modèle de probabilité génératif. Les modèles génératifs sont générallement développés suivant les 3 étapes suivantes :

1. **Observations** Les données sont considérées comme des observations générées par le modèles. La notion de données inclue les variables latenntes. Ces dernières représentent la structure thématique du corpus.
2. **Apprentissage** On met à jour les variables du modèles en utilisant l'inférence à postériori
3. **Inférence** On peut utiliser le modèle pour situer de nouvelles de données dans la structure de topics apprise.

L'originalité de la LDA réside dans sa modélisation de chaque document comme une distribution de topics. La plupart des modèles considérant que tous les mots d'un document sont issus du même topic. Ainsi chaque document peut être rattaché à plusieurs topics. Dans notre corpus exemple, la LDA pourrait générer une distribution du type :

* Document 1 : 50% Topic "Transports" + 50% Topic "Voyages"
* Document 2 : 50% Topic "Sports" + 50% Topic "Voyages"
* Document 3 : 100% Topic "Sports"
* Document 4 : 100% Topic "Transports"

Comme illustré précédemment, la LDA cherche à expliquer ces modèles en introduisant une variable latente : les topics.

<img src="./figures/lda2.png" width="1000">

### Notations

Les modèles graphiques représentent les variables aléatoies comme des noeuds. Les arcs entre les noeuds indiquent les variables potentiellement dépendantes. Les variables observées sont grisées. Dans la figure ci-dessous, les noeuds $ X_{1,...,N}$ sont observés alors que le noeud $Y$ est une variable latente. Les variables observées dépendent de cette variable latente. Les rectangles synthétisent la réplication de plusieurs structure. Un rectangle résume donc plusieurs variables $X_n$ avec $n \in N$.

La structure du graph définie les dépendances conditionnelles entre l'ensemble des variables. Par exemple dans le graph ci-dessous, on a $p(Y,X_{1},...,X_{N})=p(Y)\prod _{n=1}^{N}p(X_{n}|Y)$.

<img src="./figures/graphical_model.png" width="500">

### La distribution de Dirichlet


La distribution de Dirichlet est généralement notée $Dir(\alpha)$. Il s'agit d'une famille de lois de probabilité continues pour des variables aléatoires multinomiales. Cette loi (ou encore distribution) est paramétrée par le vecteur ${\bf \alpha}$ de nombres réels positifs. La taille du vecteur ${\bf \alpha}$ indique la dimension de la distribution. Ce type de distribution est souvent utilisée comme distribution à priori dans les modèles Bayésiens. Sans rentrer dans les détails, voici quelques caraactéristiques de la distribution de Dirichlet :

* La distribution est définie sur un simplex de vecteurs positifs dont la somme est égale à 1 
* Sa densité est caractérisée par : $P(\theta |{\overrightarrow {\alpha }})={\frac {\Gamma (\Sigma _{i}\alpha _{i})f}{\Pi _{i}\Gamma (\alpha _{i})}}\Pi _{i}\theta _{i}^{\alpha _{i}-1}$
* En pratique, si toutes les dimensions de ${\bf \alpha}$ ont des valeurs similaires, la distribution est plus étalée. Elle devient plus concentrée pour des valeurs plus importantes de ${\bf \alpha}$.

La distribution est illustrée ci-dessous pour des valeurs ${\bf \alpha}$ qui varient entre (6, 2, 2), (3, 7, 5), (6, 2, 6), (2, 3, 4).

<img src="./figures/dirichlet.png" width="500">

On utilise les notations suivantes :

* Les mots sont l'unité de base des données. Ils sont définis comme un élément d'un vocabulaire indexé par ${1,...,V}$. On erprésent chaque mot en utilisant des vecteurs one-hot dont toutes les composantes sont 0, sauf pour la composante qui correspond à l'index du mot, qui vaut 1.
* Un document est une séquence de $N$ mots telle que $W=(w_{1},w_{2},...,w_{N})$ avec $w_{n}$ le $n$th mot de la séquence.
* Un corpus est un ensemble de $M$ documents tels que $D=\lbrace W_{1},W_{2},...,W_{M}\rbrace$.

### Le principe de la LDA

La LDA est un modèle génératif. L'idée de baase est que chaque document est représenté comme une distribution sur $k$ topics latents. Chaque topic est caractérisé par une distribution sur les mots. On appelle $\beta$ la matrice de dimension $k*V$ qui représente la distribution des mots sur les $k$ topics. On a ainsi $\beta _{ij}=p(w_{j}=1|z_{i}=1)$.

La LDA suppose le procéssus génératif suivant pour chaque document $W$ dans le corpus $D$.


> 1. On choisit $\theta \sim Dir(\alpha )$
> 2. Pour chaque document dans le corpus:
>     * Pour chacun des $N$ mots $w_{n}$ dans le document :
>        * on choisit un topic $z_{n}\sim Multinomial(\theta )$
>        * on choisit un mot $w_{n}$ $p(w_{n}|z_{n},B)$ selon une loi multinimoale conditionnée par le topic $z_{n}$.


Une variable aléatoire suivant une loi de dirichlet de dimension $k$ peut prendre des valeurs dans le $k-1$-simplex. Cet espace désigne les vecteurs $\theta$ de dimension $k$ tels que $\theta _{i}\geq 0$ et $\sum _{i=1}^{k}\theta _{i}=1$

Etant donné $\alpha$ et $\beta$, la probabilité jointe de $\theta$, un ensemble de $K$ topics $Z$ et un ensemble de $N$ mots est donnée par : $$p(\theta ,Z,W|\alpha ,\beta )=p(\theta |\alpha )\prod _{n=1}^{N}p(z_{n}|\theta )p(w_{n}|z_{n},\beta ),$$

Avec $p(z_{n}|\theta )$ qui représente $\theta _{i}$ pour chaque $i$ tel que $z_{i}^{n}=1$.

On peut obtenir la distribution marginale du document en itérant sur les $\theta$ et en sommant sur les $z$:

$$p(W|\alpha ,\beta )=\int p(\theta |\alpha ){\big (}\prod _{n=1}^{N}\Sigma _{z_{n}}p(z_{n}|\theta )p(w_{n}|z_{n},\beta {\big )}d\theta$$

Finalement, en prenant le produit des probabilitées marginales sur un chaque document, on obtient la probabilité du corpus :


$$p(D|\alpha ,\beta )=\prod _{d=1}^{M}\int p(\theta _{d}|\alpha ){\big (}\prod _{n=1}^{N_{d}}\Sigma _{z_{dn}}p(z_{dn}|\theta _{d})p(w_{dn}|z_{dn},\beta ){\big )}d\theta _{d}$$

La représentation garphique du modèle est proposée ci-dessous.

<img src="./figures/lda_graph.png" width="700">

Avec les notations suivantes :
* k — Le nombre de topics qui caractérisent un document
* V — La taille du vocabulaire
* M — Le nombre de documents
* N — Le nombre de mots dans chaque document
* w — Un mot dans un document représenté par un vecteur one-hot de taille V
* **w** — Représentation d'un document, i.e. une collection de N vecteurs w de mots
* D — Le corpus, i.e. une collection de M documents
* z — Un topic parmi k. Un topic correspond à une distribution de mots. Par exemple, Animal = (0.3 Chats, 0.4 Chiens, 0.2 Adorable)

## 2. Apprendre les paramètres du modèle

Ce modèle permet d'expliquer comment le corpus a été généré. Néanmoins en pratique on n'observe pas toutes les variables du modèles mais seulement la distribution des mots. En pratique on va "inverser" le modèle pour estimer les paramètres. Ce processus est appelé inférence à posteriori.

On dispose d'un corpus de documents. On a fixé un nombre $k$ de topics. Pour apprendre la représentation des topics et des documents, il existe deux méthodes principales :

* Le sampling de Gibbs
* L'inférence variationelle

Nous allons détailler le procéssus du sampling de Gibbs sur notre jeu de données exemples puis utiliser la librarie `sklearn`qui repose sur l'inférence variationelle pour expérimenter sur un jeu de données plus conséquent.

In [76]:
from tqdm import tqdm

# IPython automatically reload all changed code
%load_ext autoreload
%autoreload 2

In [56]:
n_topics = 3

### Le Sampling de Gibbs

Pour fixer les paramètres des matrices, on cherche à maximiser la vraisemblance de nos données selon ce modèle. On utilise pour cela l'algorthime de **sampling de Gibbs**. Le sampling de Gibbs est un algorithme qui permet de sélectionner des distributions conditionelles dont la distribution des états converge vers la vraie distribution à terme. En pratique, on va mettre à jour itérativement les matrices $\Theta$ et $\beta$ pour maximiser la vraisemblance de nos données. Les itérations s'effectuent mot par mot en ajustant le topic assigné à chaque mot.  On fait l'hypothèse que l'on ne connait pas le topic assigné à chaque mot mais qu'on connait la correpondance de topic pour tous les autres mots dans le texte et on cherche à inférer le topic à assigner pour ce mot.


D'un point de vue mathématique, on cherche la probabilité conditionnelle pour chaque mot d'être assigné à un topic, étant donné l'attribution des autres topics. 

On peut montrer que 

$$P(z_{i,d}=k|z_{:,d},w,\alpha,\beta) \propto \frac{n_{d,k}+\beta_k}{\sum_i^Kn_{d,i}+\beta_i}v_{k,w_{d,n}}+\alpha_{w_{d,n}}$$

Avec :
* $n_{d,k}$ : le nombre de fois ou le document $d$ utilise le topic $k$
* $v_{k,w}$ : le nombre de fois ou le topic $k$ utilise le mot $w$
* $\alpha_k$ : le paramètre de dirichlet pour la distribution de topics par documents
* $\lambda_w$ : Le paramètre de dirichlet pour la distribution des mots par topic

Il y a deux parties dans cette équation. D'abord on évalue la proprtion de chaque topic dans un document. Puis on réparti l'attention des topics pour chaque mot. Les paramètres de dirichlet permette de régulariser quand $n_{d,k}$ ou $v_{k,w}$ sont trop proches de 0 et qu'il y a peu de chance qu'un mot choisise un topic.

In [239]:
#  1. Go through each document, and randomly assign each word in the document to one of the topics 

words_topic = [[np.random.randint(n_topics) for _ in range(len(d))] for d in documents]

print("assignation de topic aléatoire pour le premier document")
for (w, t) in zip(documents[0], words_topic[0]):
    print("{:>10} -> TOPIC {:1}".format(w, t))
    
print("matrices d'assignation des topics :")
words_topic

assignation de topic aléatoire pour le premier document
      nous -> TOPIC 1
     avons -> TOPIC 1
      pris -> TOPIC 2
     avion -> TOPIC 0
      pour -> TOPIC 0
     aller -> TOPIC 2
        en -> TOPIC 0
  vacances -> TOPIC 2
matrices d'assignation des topics :


[[1, 1, 2, 0, 0, 2, 0, 2],
 [2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 1, 1, 2, 0],
 [2, 0, 0, 1]]

* $n_{d,k}$ : le nombre de fois ou le document $d$ utilise le topic $k$
* $v_{k,w}$ : le nombre de fois ou le topic $k$ utilise le mot $w$

In [240]:
# 2. Notice that this random assignment already gives both topic representations 
#    of all the documents and word distributions of all the topics (albeit not very good ones).

document_topic_counts = np.zeros((len(documents), n_topics), dtype=int)
for (doc_idx, topics) in enumerate(words_topic):
    topics, topics_freq = np.unique(topics, return_counts=True)
    for (t, f) in zip(topics, topics_freq):
        document_topic_counts[doc_idx][t] = f
print("Le nombre de fois ou le document  𝑑  utilise le topic  𝑘 :")
print(document_topic_counts, '\n')

topic_word_counts = np.zeros((n_topics, vocab_size + 1), dtype=int)
for (topics, doc) in zip(words_topic, documents):
    for (t, w_idx) in zip(topics, doc):
        topic_word_counts[t][w2idx[w_idx]] += 1
print("Le nombre de fois ou le topic  𝑘  utilise le mot  𝑤 :")
print(topic_word_counts)

Le nombre de fois ou le document  𝑑  utilise le topic  𝑘 :
[[3 2 3]
 [3 2 9]
 [2 1 1]] 

Le nombre de fois ou le topic  𝑘  utilise le mot  𝑤 :
[[0 0 0 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0]
 [0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0]
 [2 1 0 0 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 2 0]]


In [241]:
# nombre de mots pour chaque topic:
np.sum(topic_word_counts, 1)

array([ 8,  5, 13])

In [242]:
# 3. So to improve on them, for each document W do the following:
#    3.1 Go through each word w in W
#        3.1.1 And for each topic t, compute two things: 
#              1) p(t|W) = the proportion of words in document W that are currently assigned to topic t, and 
#              2) p(w|t) = the proportion of assignments to topic t over all documents that come from this word w 
#              Reassign w a new topic, where we choose topic t with probability p(t|W)*p(w|t). 
#              It is worth noting that according to our generative model, 
#              this is essentially the probability that topic t generated word w, 
#              so it makes sense that we resample the current word’s topic with this probability. 
#              (Also, I’m glossing over a couple of things here, 
#              in particular the use of priors/pseudocounts in these probabilities.)
#        3.1.2 In other words, in this step, we’re assuming that all topic assignments 
#              except for the current word in question are correct, and then updating the assignment 
#              of the current word using our model of how documents are generated.
eta = 0.01
alpha = 50.


for (doc_idx, doc) in enumerate(documents):
    for (w_idx, w) in enumerate(doc):
        
        
        current_topic = words_topic[doc_idx][w_idx]
        
        # Mise à jour des matrices de fréquences
        document_topic_counts[doc_idx, current_topic] -= 1
        topic_word_counts[current_topic, w2idx[w]] -= 1
        
        # Change le topic
        topic_distribution = (topic_word_counts[:, w2idx[w]] + eta) * \
            (document_topic_counts[doc_idx, :] + alpha) / \
            (np.sum(topic_word_counts, 1) + 1e-12)

        new_topic = np.random.multinomial(1, topic_distribution / topic_distribution.sum()).argmax()
        
        # Mise à jour des matrices de fréquences
        document_topic_counts[doc_idx][new_topic] += 1
        topic_word_counts[new_topic, w2idx[w]] += 1
        words_topic[doc_idx][w_idx] = new_topic

In [243]:
document_topic_counts

array([[ 5,  3,  0],
       [ 2, 12,  0],
       [ 3,  1,  0]])

In [244]:
topic_word_counts

array([[1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [252]:
# 4. After repeating the previous step a large number of times, 
#    you’ll eventually reach a roughly steady state where your assignments are pretty good. 
#    So use these assignments to estimate the topic mixtures of each document 
#    (by counting the proportion of words assigned to each topic within that document) 
#    and the words associated to each topic (by counting the proportion of words assigned to each topic overall).

n_iters = 10

for _ in range(n_iters):
    for (doc_idx, doc) in enumerate(documents):
        for (w_idx, w) in enumerate(doc):


            current_topic = words_topic[doc_idx][w_idx]

            # Mise à jour des matrices de fréquences
            document_topic_counts[doc_idx, current_topic] -= 1
            topic_word_counts[current_topic, w2idx[w]] -= 1

            # Change le topic
            topic_distribution = (topic_word_counts[:, w2idx[w]] + eta) * \
                (document_topic_counts[doc_idx, :] + alpha) / \
                (np.sum(topic_word_counts, 1) + 1e-12)

            new_topic = np.random.multinomial(1, topic_distribution / topic_distribution.sum()).argmax()

            # Mise à jour des matrices de fréquences
            document_topic_counts[doc_idx][new_topic] += 1
            topic_word_counts[new_topic, w2idx[w]] += 1
            words_topic[doc_idx][w_idx] = new_topic

In [253]:
document_topic_counts

array([[4, 4, 0],
       [7, 7, 0],
       [1, 3, 0]])

Finalement on peut évaluer les matrices $\Theta$ et $\beta$ selon les formules suivantes :

$$\Theta_{k,t} = \frac{n_{d,k}+\eta_k}{\sum_i^Kn_{d,i}+\eta_k}$$

$$\beta_{m,k} = \frac{v_{k}+\alpha_k}{\sum_i^Kn_{d,i}+\alpha_k}$$


In [256]:
beta = (topic_word_counts + alpha) / (np.sum(topic_word_counts, 0) + alpha)

In [255]:
n_top_words = 3

for topic_idx in range(n_topics):
    message = "Topic #%d: " % topic_idx
    message += " ".join([idx2w[i] for i in beta[topic_idx].argsort()[:-n_top_words - 1:-1][1:]])
    print(message)

Topic #0: matin aller
Topic #1: hier taxi
Topic #2: la aller


## 3. Utilisation des librairies

In [316]:
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
import pysrt
import re
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS
import unidecode

On va chercher à analyser les thèmes de la Série Game Of Thrones. On utilise pour ça les sous-titres de l'ensemble des saisons qui ont été récupérés sur le site https://www.sous-titres.eu/series/game_of_thrones.html.

In [258]:
def create_subtitle_file_dict(subtitles_dir):
    "Retourne les chemins vers les fichiers de sous titres"
    subtitles_file_path = {}
    for path, subdirs, files in os.walk(subtitles_dir):
        for name in files:
            episode_name = '_'.join([os.path.basename(path), name.split('.')[0]])
            subtitles_file_path[episode_name] = os.path.join(path, name)
    return subtitles_file_path

def parse_srt_file(srt_file, encoding='iso-8859-1'):
    "Lit un ficher de sous titres au format rst"
    subs = pysrt.open(srt_file, encoding=encoding)
    text = ' '.join([' '.join(sub.text.split('\n')) for sub in subs])
    return text

def create_corpus(subtitles_file_path):
    "Créer un corpus à partir de tous les fichiers rst dans un dossier"
    corpus = []
    for k, v in subtitles_file_path.items():
        if v.endswith('srt'):
            corpus.append(parse_srt_file(v))
    return corpus

In [259]:
subtitles_file_path = create_subtitle_file_dict('./sous-titres-got')

In [340]:
episode_1_txt = parse_srt_file(subtitles_file_path['S01_E01'])

In [341]:
print(episode_1_txt[:100])

Doucement. Que croyais-tu ? Ce sont des sauvages. L'un vole une chèvre à l'autre, et ils finissent p


In [303]:
corpus = create_corpus(subtitles_file_path)

In [304]:
len(corpus)

73

In [342]:
corpus[0][:100]

'attention au feu au feu une vingtaine d hommes voire moins ils se sont faufiles dans le camp ils ont'

<div class="alert alert-info" role="alert">
    <p><b>Exercice :</b> Nettoyer le corpus pour enlever les accents, mettre le texte en minuscule, enlever la ponctuation et les doubles espaces. Eventuellement pour le stemming </p>
</div>    

In [ ]:
# %load solutions/cleaning.py


In [344]:
clean_corpus = clean_corpus(corpus)

In [345]:
clean_corpus[0][:100]

'attent au feu au feu une vingtain d homm voir moin il se sont faufil dan le camp il ont brul nos viv'

In [346]:
clean_corpus_split = []
for episode in clean_corpus:
    episode_words = episode.split()
    i = 0
    while i < len(episode_words):
        clean_corpus_split.append(' '.join(episode_words[i:i+400]))
        i+=400

In [347]:
len(clean_corpus_split)

726

In [348]:
def tokenize_corpus(corpus):
    tokens = []
    for sentence in corpus.split('\n'):
        tokens.append(nltk.word_tokenize(sentence))
    return tokens

In [349]:
sentence_length = [len(x.split()) for x in clean_corpus_split]

In [350]:
np.mean(sentence_length), np.std(sentence_length)

(378.10743801652893, 75.55046495285958)

<div class="alert alert-info" role="alert">
    <p><b>Exercice :</b> Vectorizer le corpus en utilisant la méthode Bag-Of-Words.</p>
</div>    

In [351]:
# %load solutions/vectorize.py


In [352]:
len(clean_corpus_split)

726

In [353]:
# Tweak the two parameters below
number_topics = 15
number_words = 10

# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)

LatentDirichletAllocation(n_components=15, n_jobs=-1)

In [354]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [355]:
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
qu reine oui ca avez ici jamais mort sais majeste

Topic #1:
stark pere lord qu fille batard nord garcon bolton mere

Topic #2:
nord qu jon winterfell snow stark mur ca hommes morts

Topic #3:
qu roi homme hommes veux majeste reine joffrey dragons sang

Topic #4:
roi qu robert stannis garde bon epouse ca lord faire

Topic #5:
qu avez nord roi ca lord jon sansa lannister etes

Topic #6:
qu ca veux roi pere faut mort faire sais ete

Topic #7:
qu etes avez reine ca pere hommes faire homme port

Topic #8:
qu ca etes sais jamais nuit ici veux faire avez

Topic #9:
epee ca qu veux mort petite faire ici acier tue

Topic #10:
avez roi grand reine qu tyrell loras faire frere mere

Topic #11:
qu dubbing adaptation brothers titrage ca pere etes faire ici

Topic #12:
lannister pere roi qu avez fils stark ca oui lord

Topic #13:
roi qu lord fils stark pere ser hommes main frere

Topic #14:
qu etes brienne lady compte sansa avez fille stark oui


## 4. Visualisation

In [356]:
# !pip install pyLDAvis

In [357]:
%%time
from pyLDAvis import sklearn as sklearn_lda
import pickle
import pyLDAvis

LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(number_topics))
LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer, mds='mmds')

CPU times: user 218 ms, sys: 20 ms, total: 238 ms
Wall time: 1.11 s


In [358]:
with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(number_topics) +'.html')

In [359]:
pyLDAvis.display(LDAvis_prepared)

<span class="badge badge-secondary">(Sievert et al., 2014)</span> Sievert, Carson, and Kenneth Shirley. "LDAvis: A method for visualizing and interpreting topics." Proceedings of the workshop on interactive language learning, visualization, and interfaces. 2014.

<span class="badge badge-secondary">(Chuang et al., 2012)</span> Chuang, Jason, Christopher D. Manning, and Jeffrey Heer. "Termite: Visualization techniques for assessing textual topic models." Proceedings of the international working conference on advanced visual interfaces. 2012.

<div class="alert alert-info" role="alert">
    <p><b>Exercice :</b> Faire varier le paramètre Lambda et justifier de son impact.</p>
</div>    

<div class="alert alert-info" role="alert">
    <p><b>Exercice :</b> Faire varier le préprocessing,en particulier la stemmatization. Analyser l'impact sur l'analyse des clusters.</p>
</div>    

<div class="alert alert-info" role="alert">
    <p><b>Exercice :</b> Etudier l'impact des Stop Words sur les topics.</p>
</div>    

Sources :
* https://medium.com/analytics-vidhya/topic-modeling-using-lda-and-gibbs-sampling-explained-49d49b3d1045
* https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158
* https://wiki.ubc.ca/Course:CPSC522/Latent_Dirichlet_Allocation
* http://www.arbylon.net/publications/text-est2.pdf

Icons made by <a href="https://www.flaticon.com/authors/freepik" title="Freepik">Freepik</a> and <a href="https://www.flaticon.com/authors/pixel-perfect" title="Pixel perfect">Pixel perfect</a> from <a href="https://www.flaticon.com/" title="Flaticon"> www.flaticon.com</a>